In [ ]:
import librosa                    
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import os
from google.colab import drive
from librosa.core import istft
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def doit(y):
  if y.shape[0] < 65500:
        y = np.append(y, np.full( (65500-y.shape[0]), np.mean(y)))
  melSpec = librosa.feature.melspectrogram(y=y[:65500], sr=16000, n_mels=128)
  melSpec_dB = librosa.power_to_db(melSpec, ref=np.max)
  return melSpec_dB

In [ ]:
class AudioAugmentation:
    def read_audio_file(self, file_path):
        input_length = 65500
        data = librosa.core.load(file_path)[0]
        if len(data) > input_length:
            data = data[:input_length]
        else:
            data = np.pad(data, (0, max(0, input_length - len(data))), "constant")
        return data
    def pitch_shift(self,data):
        data=librosa.effects.pitch_shift(data, sr=16000, n_steps=3,bins_per_octave=24)
        # if len(data) > input_length:
        #     data = data[:input_length]
        # else:
        #     data = np.pad(data, (0, max(0, input_length - len(data))), "constant")
        # return doit(data)
        return data
    def add_noise(self, data):
        noise = np.random.randn(len(data))
        data_noise = data + 0.005 * noise
        return data_noise
    def shift(self, data,count):
        return (np.roll(data,count))
        
    def stretch(self, data, rate):
        input_length = 65500
        #data=istft(data)
        data = librosa.effects.time_stretch(data, rate)
        # if len(data) > input_length:
        #     data = data[:input_length]
        # else:
        #     data = np.pad(data, (0, max(0, input_length - len(data))), "constant")
        # return doit(data)
        return data
    def write_audio_file(self, file, data, sample_rate=16000):
        librosa.output.write_wav(file, data, sample_rate)
    def plot_time_series(self, data):
        fig = plt.figure(figsize=(14, 8))
        plt.title('Raw wave ')
        plt.ylabel('Amplitude')
        plt.plot(np.linspace(0, 1, len(data)), data)
        plt.show()

In [ ]:
def get_cat(file_name):
    for i in range(len(file_name)):
        if(file_name[i] == '-'):
            return int(file_name[i+1])

In [ ]:
# formatting the binary way 
# 0 to 7.
aa = AudioAugmentation()
base_dir = '/content/drive/My Drive/sem5MlProject/dataset/audio/'
x=0;
binary=[[0,0,0,0,0,0],[1,0,1,1,0,0],[0,1,0,1,0,0],[0,0,0,0,1,0],[0,0,0,0,0,1]] # speedUp speedDown pitchShift addNoise roll
for fold in os.listdir(base_dir):
  if fold!='fold9' and fold!='fold7':
    continue
  datalist=[]
  for audio_file in os.listdir(base_dir+fold):
    for l in binary:
      data=aa.read_audio_file(base_dir+fold+'/'+audio_file)
      if l[0]==1:
        data=aa.stretch(data,0.9)
      if l[1]==1:
        data=aa.stretch(data,1.1)
      if l[2]==1:
        data=aa.pitch_shift(data)
      if l[3]==1:
        data=aa.add_noise(data)
      if l[4]==1:
        data=aa.shift(data,5000)
      if l[5]==1:
        data=aa.shift(data,10000)    
      data=doit(data)
      datalist.append([data,get_cat(audio_file)])
    x=x+1
    if x%100==0:
      print(x)
    #aa.plot_time_series(data)
    #aa.write_audio_file('/content/drive/My Drive/sem5MlProject/dataset/Mohnish_augumentation/'+fold+audio_file,data)
  np.save('/content/drive/MyDrive/sem5MlProject/dataset/PreProcessedData/augumentationALL_Mohnish3/'+fold+'.npy', datalist)
  print(fold+' done')


100
200
300
400
500
600
700
800


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


fold7 done
900
1000
1100
1200
1300
1400
1500
1600
fold9 done
